# einops 自学入门

## 常见问题

### 1. 为什么叫einops？

ein的意思是爱因斯坦（Einstein），ops的意思是操作（operations）。

意思是这个操作是爱因斯坦发明的。

### 2. einops 是用来干啥的？

是用来简化tensor操作的。例如，要把 `x = [b,h,w,c]` 改成 `x = [b,c,h,w]`。

In [6]:
import torch
b = 8
c = 3
h = 100
w = 200
x = torch.randn(b, h , w, c)
print(x.shape)

torch.Size([8, 100, 200, 3])


原来的操作应该是 `x = x.permute(0,3,1,2)`，要数这个1234就很麻烦。

用了einops以后，写法就变成 `x = rearrange(x, 'b h w c -> b c h w')`，更直观，不用再数数了。

In [8]:
x1 = x.permute(0,3,1,2)
print(x1.shape)

from einops import rearrange
x2 = rearrange(x,'b h w c -> b c h w')
print(x2.shape)

torch.Size([8, 3, 100, 200])
torch.Size([8, 3, 100, 200])



## 常用的einops方法

### 1. 利用 pack 和 unpack 处理 ViT 里的 class token

ViT里，patch token [b, n, d] 和 class token [b, d] 需要串起来，得到 [b n+1 d]。

经过 transformer 以后，再取出 class token 用于后续的处理。

In [1]:
import torch
from einops import pack, unpack
b = 8
d = 256
h = w = 16
patch_tokens = torch.randn(b, h, w, d)
class_tokens = torch.randn(b, d)
tokens, ps = pack([class_tokens, patch_tokens], 'b * d')
class_tokens, patch_tokens = unpack(tokens, ps, 'b * d')
print(tokens.shape)
print(class_tokens.shape)
print(ps)

torch.Size([8, 257, 256])
torch.Size([8, 256])
[torch.Size([]), torch.Size([16, 16])]


### 2. 维度变换和调整

张量经常调换维度的顺序，或者把几个维度合并，或者分开，都可以用 einops 里的 rearrange。

但是需要注意，字符串里的符号不能有重复的。

In [14]:
from einops import rearrange
b = 10
c = 3
h = 100
w = 200
p = 20
img = torch.randn(b,c,h,w)
patches = rearrange(img, 'b c (h ph) (w pw) -> b c h w ph pw', ph = p, pw = p)
print(patches.shape)

torch.Size([10, 3, 5, 10, 20, 20])
